In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv',header=0)
df.head()
'''label_map = pd.read_json(f'/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 
                         orient='index')'''
dict_label={
  0: "Cassava Bacterial Blight (CBB)",
  1: "Cassava Brown Streak Disease (CBSD)",
  2: "Cassava Green Mottle (CGM)",
  3: "Cassava Mosaic Disease (CMD)",
  4: "Healthy"
}

In [ ]:
df['label'].replace(dict_label, inplace=True)

In [ ]:
df.head()

In [ ]:
df.groupby('label').count()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, stratify=df.label)

In [ ]:
train.groupby('label').count()

In [ ]:
test.groupby('label').count()

In [ ]:
'''#applying categorical encoding for labels
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
train_arr = ct.fit_transform(train)
test_arr = ct.transform(test)
convert_type = {'label1':int,'label2':int,'label3':int,'label4':int,'label5':int,'image_id':object}
train=pd.DataFrame(train_arr, columns=['label1','label2','label3','label4','label5','image_id']).astype(convert_type)
test=pd.DataFrame(test_arr, columns=['label1','label2','label3','label4','label5','image_id']).astype(convert_type)'''

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
from keras.utils import to_categorical

training_set = train_datagen.flow_from_dataframe(train,directory='/kaggle/input/cassava-leaf-disease-classification/train_images',
                                                 x_col="image_id", y_col="label",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_dataframe(test,directory='/kaggle/input/cassava-leaf-disease-classification/train_images',
                                            x_col="image_id", y_col="label",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
# Building the CNN
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=512, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))

cnn.summary()

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3
)
# Training the CNN on the Training set and evaluating it on the Test set
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[es])
history.history['val_loss']

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg', 
                            target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(training_set.class_indices)
print(result)